In [41]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from sqlalchemy import create_engine
import pymongo

In [3]:
# Bring ACCIDENT CSV into a separate data frame
accident_df = pd.read_csv("resources\ACCIDENT.csv")
accident_df.head(2)

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70


In [40]:
# Data frame Column Check
accident_df.count()

ACCIDENT_NO             203708
ACCIDENTDATE            203708
ACCIDENTTIME            203708
ACCIDENT_TYPE           203708
Accident Type Desc      203708
DAY_OF_WEEK             203708
Day Week Description    203708
DCA_CODE                203708
DCA Description         203708
DIRECTORY               201360
EDITION                 201360
PAGE                    201360
GRID_REFERENCE_X        201360
GRID_REFERENCE_Y        201360
LIGHT_CONDITION         203708
Light Condition Desc    203708
NODE_ID                 203708
NO_OF_VEHICLES          203708
NO_PERSONS              203708
NO_PERSONS_INJ_2        203708
NO_PERSONS_INJ_3        203708
NO_PERSONS_KILLED       203708
NO_PERSONS_NOT_INJ      203708
POLICE_ATTEND           203708
ROAD_GEOMETRY           203708
Road Geometry Desc      203708
SEVERITY                203708
SPEED_ZONE              203708
dtype: int64

In [4]:
# Bring ACCIDENT_EVENT CSV into a separate data frame
accident_event_df = pd.read_csv("resources\ACCIDENT_EVENT.csv")
accident_event_df.head(2)

,ACCIDENT_NO,EVENT_SEQ_NO,EVENT_TYPE,Event Type Desc,VEHICLE_1_ID,VEHICLE_1_COLL_PT,Vehicle 1 Coll Pt Desc,VEHICLE_2_ID,VEHICLE_2_COLL_PT,Vehicle 2 Coll Pt Desc,PERSON_ID,OBJECT_TYPE,Object Type Desc
0,T20060000010,1.0,C,Collision,B,2,Right side (forwards),A,F,Front,,99.0,Not Applicable
1,T20060000018,1.0,C,Collision,B,F,Front,A,9,Not known or Not Applicable,,99.0,Not Applicable


In [39]:
# Data frame Column Check
accident_event_df.count()

ACCIDENT_NO               326638
EVENT_SEQ_NO              326634
EVENT_TYPE                326634
Event Type Desc           326638
VEHICLE_1_ID              326634
VEHICLE_1_COLL_PT         326634
Vehicle 1 Coll Pt Desc    326638
VEHICLE_2_ID              326634
VEHICLE_2_COLL_PT         326634
Vehicle 2 Coll Pt Desc    326638
PERSON_ID                 326634
OBJECT_TYPE               326634
Object Type Desc          326638
dtype: int64

In [33]:
# Combine ACCIDENT and ACCIDENT EVENT data frames
combined_accident_event_df = pd.merge(accident_df, accident_event_df,  how='outer', on='ACCIDENT_NO')
combined_accident_event_df.head(2)

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,Event Type Desc,VEHICLE_1_ID,VEHICLE_1_COLL_PT,Vehicle 1 Coll Pt Desc,VEHICLE_2_ID,VEHICLE_2_COLL_PT,Vehicle 2 Coll Pt Desc,PERSON_ID,OBJECT_TYPE,Object Type Desc
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,Collision,B,2,Right side (forwards),A,F,Front,,99.0,Not Applicable
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,Collision,B,F,Front,A,9,Not known or Not Applicable,,99.0,Not Applicable


In [42]:
# Data frame Column Check
combined_accident_event_df.count()

ACCIDENT_NO               326638
ACCIDENTDATE              326638
ACCIDENTTIME              326638
ACCIDENT_TYPE             326638
Accident Type Desc        326638
DAY_OF_WEEK               326638
Day Week Description      326638
DCA_CODE                  326638
DCA Description           326638
DIRECTORY                 322676
EDITION                   322676
PAGE                      322676
GRID_REFERENCE_X          322676
GRID_REFERENCE_Y          322676
LIGHT_CONDITION           326638
Light Condition Desc      326638
NODE_ID                   326638
NO_OF_VEHICLES            326638
NO_PERSONS                326638
NO_PERSONS_INJ_2          326638
NO_PERSONS_INJ_3          326638
NO_PERSONS_KILLED         326638
NO_PERSONS_NOT_INJ        326638
POLICE_ATTEND             326638
ROAD_GEOMETRY             326638
Road Geometry Desc        326638
SEVERITY                  326638
SPEED_ZONE                326638
EVENT_SEQ_NO              326634
EVENT_TYPE                326634
Event Type

In [6]:
# Bring ACCIDENT_LOCATION CSV into a separate data frame
accident_location_df = pd.read_csv("resources\ACCIDENT_LOCATION.csv")
accident_location_df.head(2)

,ACCIDENT_NO,NODE_ID,ROAD_ROUTE_1,ROAD_NAME,ROAD_TYPE,ROAD_NAME_INT,ROAD_TYPE_INT,DISTANCE_LOCATION,DIRECTION_LOCATION,NEAREST_KM_POST,OFF_ROAD_LOCATION
0,T20060000010,43078,2090.0,FOSTER,STREET,MCCRAE,STREET,0.0,SW,NaN,NaN
1,T20060000018,29720,5057.0,HALLAM,ROAD,BELGRAVE-HALLAM,ROAD,70.0,S,NaN,NaN


In [43]:
# Data frame Column Check
accident_location_df.count()

ACCIDENT_NO           203708
NODE_ID               203708
ROAD_ROUTE_1          202878
ROAD_NAME             202702
ROAD_TYPE             200309
ROAD_NAME_INT         201886
ROAD_TYPE_INT         200211
DISTANCE_LOCATION     202878
DIRECTION_LOCATION    202878
NEAREST_KM_POST           45
OFF_ROAD_LOCATION      21924
dtype: int64

In [9]:
# Combine ACCIDENT and ACCIDENT EVENT combined data frames with ACCIDENT LOCATION data frame
combined_accident_df =pd.merge(combined_accident_event_df, accident_location_df, how='outer', on='ACCIDENT_NO')
combined_accident_df.head(2)

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NODE_ID_y,ROAD_ROUTE_1,ROAD_NAME,ROAD_TYPE,ROAD_NAME_INT,ROAD_TYPE_INT,DISTANCE_LOCATION,DIRECTION_LOCATION,NEAREST_KM_POST,OFF_ROAD_LOCATION
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,43078,2090.0,FOSTER,STREET,MCCRAE,STREET,0.0,SW,NaN,NaN
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,29720,5057.0,HALLAM,ROAD,BELGRAVE-HALLAM,ROAD,70.0,S,NaN,NaN


In [35]:
# Data frame Column Check
combined_accident_df.count()

ACCIDENT_NO               326638
ACCIDENTDATE              326638
ACCIDENTTIME              326638
ACCIDENT_TYPE             326638
Accident Type Desc        326638
DAY_OF_WEEK               326638
Day Week Description      326638
DCA_CODE                  326638
DCA Description           326638
DIRECTORY                 322676
EDITION                   322676
PAGE                      322676
GRID_REFERENCE_X          322676
GRID_REFERENCE_Y          322676
LIGHT_CONDITION           326638
Light Condition Desc      326638
NODE_ID_x                 326638
NO_OF_VEHICLES            326638
NO_PERSONS                326638
NO_PERSONS_INJ_2          326638
NO_PERSONS_INJ_3          326638
NO_PERSONS_KILLED         326638
NO_PERSONS_NOT_INJ        326638
POLICE_ATTEND             326638
ROAD_GEOMETRY             326638
Road Geometry Desc        326638
SEVERITY                  326638
SPEED_ZONE                326638
EVENT_SEQ_NO              326634
EVENT_TYPE                326634
Event Type

In [44]:
# Creating a filtered dataframe from specific columns.

# FEEL FREE TO ADD MORE COLUMNS!!! AS PER ERD

accident_cols =["ACCIDENT_NO", "Day Week Description", "ACCIDENTDATE", "ACCIDENTTIME", "Accident Type Desc", "DCA Description", "DIRECTORY", "ROAD_NAME", "ROAD_TYPE", "ROAD_TYPE_INT"]
accident_transformed_df= combined_accident_df[accident_cols].copy()

# setting the index..... or not
accident_transformed_df.set_index("ACCIDENT_NO", inplace=False)

accident_transformed_df.head(5)

,ACCIDENT_NO,Day Week Description,ACCIDENTDATE,ACCIDENTTIME,Accident Type Desc,DCA Description,DIRECTORY,ROAD_NAME,ROAD_TYPE,ROAD_TYPE_INT
0,T20060000010,Friday,13/01/2006,12:42:00,Collision with vehicle,RIGHT NEAR (INTERSECTIONS ONLY),MEL,FOSTER,STREET,STREET
1,T20060000018,Friday,13/01/2006,19:10:00,Collision with vehicle,RIGHT NEAR (INTERSECTIONS ONLY),MEL,HALLAM,ROAD,ROAD
2,T20060000022,Saturday,14/01/2006,12:10:00,Fall from or in moving vehicle,FELL IN/FROM VEHICLE,MEL,BROWNS,ROAD,ROAD
3,T20060000023,Saturday,14/01/2006,11:49:00,Collision with vehicle,REAR END(VEHICLES IN SAME LANE),MEL,SPRINGVALE,ROAD,AVENUE
4,T20060000026,Saturday,14/01/2006,10:45:00,Collision with vehicle,RIGHT THROUGH,MEL,ELIZABETH,AVENUE,CRESCENT


In [45]:
# Data frame Column Check
accident_transformed_df.count()

ACCIDENT_NO             326638
Day Week Description    326638
ACCIDENTDATE            326638
ACCIDENTTIME            326638
Accident Type Desc      326638
DCA Description         326638
DIRECTORY               322676
ROAD_NAME               325021
ROAD_TYPE               320865
ROAD_TYPE_INT           320811
dtype: int64

In [46]:
accident_transformed_df['DIRECTORY'].value_counts()

MEL    233580
VCS     75642
         8513
VCD      4941
Name: DIRECTORY, dtype: int64

In [47]:
accident_transformed_df['ROAD_NAME'].value_counts()

PRINCES            8704
HIGH               4497
NEPEAN             3695
MONASH             3192
SOUTH GIPPSLAND    2793
                   ... 
HERLIHYS              1
FOCH                  1
JENKENS               1
HABEL                 1
BUCKLERS              1
Name: ROAD_NAME, Length: 14030, dtype: int64

In [48]:
accident_transformed_df['ROAD_TYPE'].value_counts()

ROAD        165245
STREET       69140
HIGHWAY      31689
FREEWAY      14822
AVENUE        8553
             ...  
GATE             1
STREET S         1
CRESENT          1
VALLEY           1
GLEN             1
Name: ROAD_TYPE, Length: 83, dtype: int64

In [49]:
accident_transformed_df['ROAD_TYPE_INT'].value_counts()

ROAD        122969
STREET       97248
AVENUE       17245
DRIVE        16379
RAMP         14336
             ...  
FAIRWAY          1
TURN             1
APPROACH         1
GATE             1
HEIGHTS          1
Name: ROAD_TYPE_INT, Length: 110, dtype: int64

In [50]:
accident_transformed_df.describe()

,ACCIDENT_NO,Day Week Description,ACCIDENTDATE,ACCIDENTTIME,Accident Type Desc,DCA Description,DIRECTORY,ROAD_NAME,ROAD_TYPE,ROAD_TYPE_INT
count,326638,326638,326638,326638,326638,326638,322676,325021,320865,320811
unique,203708,7,5416,1440,9,81,4,14030,83,110
top,T20160000627,Friday,21/04/2017,16:00:00,Collision with vehicle,REAR END(VEHICLES IN SAME LANE),MEL,PRINCES,ROAD,ROAD
freq,20,51082,131,4663,191262,51588,233580,8704,165245,122969


In [19]:

accident_transformed_df.to_csv("Victorian_Accident_Data_2006-2020.csv")

In [17]:
# Postgres connection setup
# Creating database connection
# connection_string = "postgres:postgres@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{connection_string}')


In [ ]:
# Postgres connection setup
# Confirm tables
#engine.table_names()

In [ ]:
# Load DataFrame into database
# accident_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [ ]:
#Mongo setup
# The default port used by MongoDB is 27017
#conn = 'mongodb://localhost:27017'
#client = pymongo.MongoClient(conn)

# Define the database in Mongo
#db = client.accident_transformed